In [1]:
import pandas as pd
from scipy.spatial.distance import cosine

In [2]:
#Reading the four csv files in the MovieLens Downloaded data
user_cols = ['user_id', 'age', 'gender', 'occupation', 'zip_code']
users = pd.read_csv('../../../Downloads/ml-100k/u.user', sep='|', names=user_cols,encoding='latin-1')

rating_cols = ['user_id', 'movie_id','rating', 'timestamp']
ratings = pd.read_csv('../../../Downloads/ml-100k/u.data', sep='\t', names=rating_cols,encoding='latin-1')

movies_cols = ['movie_id','movie_title', 'release_date', 'video_release_date','IMDb_URL', 'unknown', 'Action', 
               'Adventure','Animation','Children', 'Comedy' , 'Crime', 'Documentary', 'Drama', 'Fantasy',
              'Film_Noir',  'Horror',  'Musical', 'Mystery', 'Romance', 'Sci-Fi','Thriller','War','Western']
movies = pd.read_csv('../../../Downloads/ml-100k/u.item', sep='|', names=movies_cols,encoding='latin-1')


In [3]:
print(users.shape)
users.head()
print(ratings.shape)
ratings.head()
print(movies.shape)
movies.head()

(943, 5)
(100000, 4)
(1682, 24)


,movie_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children,...,Fantasy,Film_Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [4]:
#Splitting the data. Utilising the data split prior by the Movie Lens itself
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings_train = pd.read_csv('../../../Downloads/ml-100k/ua.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('../../../Downloads/ml-100k/ua.test', sep='\t', names=r_cols, encoding='latin-1')
ratings_train.shape, ratings_test.shape

((90570, 4), (9430, 4))

In [67]:
ratings_train.head()

,user_id,movie_id,rating,timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [91]:
#Reshaping the ratings dataframe in the following format where the user and his preference are listed as below
#The rows are the users and the columns the movies
len(set(ratings_train['user_id']))
movies_train = ratings_train.pivot(index='user_id', columns='movie_id', values='rating')
movies_train = movies_train.fillna(0)
value = [0] * len(set(ratings_train['user_id']))
movies_train.insert(1581,'1582',value)
movies_train.insert(1652,'1653',value)
movies_train.head()


movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [93]:
# --- Start Item Based Recommendations --- #
#uses cosine similarity
# Drop all columns that do not describe the type of item.
#Create a placeholder dataframe listing item vs. item

movies_train.columns = movies['movie_title']
movie_similarity = pd.DataFrame(index=movies_train.columns,columns=movies_train.columns)


1582
1653


In [79]:
# Filling the dataframe movie_similarity with the cosine values by
# Looping through the columns
for i in range(1,len(movie_similarity.columns)) :
    # Loop through the columns for each column
    for j in range(1,len(movie_similarity.columns)) :
      # Fill in placeholder with cosine similarities
      movie_similarity.ix[i,j] = 1-cosine(movies_train.ix[:,i],movies_train.ix[:,j])
        
movie_similarity.head()

/Users/BhavikaJalli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys
/Users/BhavikaJalli/anaconda3/lib/python3.6/site-packages/scipy/spatial/distance.py:505: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - np.dot(u, v) / (norm(u) * norm(v))
/Users/BhavikaJalli/anaconda3/lib/python3.6/site-packages/scipy/spatial/distance.py:505: RuntimeWarning: invalid value encountered in true_divide
  dist = 1.0 - np.dot(u, v) / (norm(u) * norm(v))


movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
movie_id,,,,,,,,,,,,,,,,,,,,,
1,1,0.269185,0.465631,0.328416,0.0773289,0.382692,0.318898,0.252585,0.167412,0.406408,...,0,0,0,0,0,0,0.0815591,0.0815591,NaN,NaN
2,0.269185,1,0.297956,0.195019,0.1089,0.365814,0.216736,0.289609,0.164617,0.352178,...,0,0,0.0330049,0,0,0,0,0.0990148,NaN,NaN
3,0.465631,0.297956,1,0.325537,0.102273,0.489017,0.453511,0.441084,0.243466,0.570986,...,0.0966556,0.0966556,0.0386622,0,0,0,0.0579934,0.0773245,NaN,NaN
4,0.328416,0.195019,0.325537,1,0.0429511,0.345893,0.247985,0.275585,0.0603687,0.388631,...,0,0,0,0,0,0,0,0.0984798,NaN,NaN
5,0.0773289,0.1089,0.102273,0.0429511,1,0.129059,0.0975818,0.162362,0.201311,0.0742894,...,0,0,0,0,0,0,0,0,NaN,NaN


In [81]:


#With the similarity matrix filled out we can look for each items “neighbour” by looping through ‘movie_similarity’, 
#sorting each column in descending order,and grabbing the name of each of the top 10 movies.
# Create a placeholder items for closes neighbours to an item
movie_neighbours = pd.DataFrame(index=movie_similarity.columns,columns=range(1,11))
movie_neighbours.head() 
# Loop through our similarity dataframe and fill in neighbouring item names
for i in range(0,len(movie_similarity.columns)):
    movie_neighbours.ix[i,:10] = movie_similarity.ix[0:,i].sort_values(ascending=False)[:10].index

movie_neighbours.head()    
#End of Item Based Filtering

ValueError: Length mismatch: Expected axis has 1681 elements, new values have 1682 elements